In [1]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('whatever.vol')
mesh.Curve(2)
# mesh.Refine()

In [2]:
%run J.ipynb

using 3d mesh with ne= 21022  elements and nv= 3727  points
Solving...
... Success!
Number of iterations = 396/1000 | Residual = 9.688436042944872e-06
999.9999999999955
Solving...
... Success!
Number of iterations = 321/1000 | Residual = 9.965986531095817e-06
1000.00109446151


In [3]:
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"

BH data from TEAM 13 problem
returning energy


In [8]:
p = 1

Q = ngs.VectorL2(mesh, order = p-1)
V = ngs.H1(mesh, order = p, dirichlet = "ambient_face")

X = Q*V
(p,u), (q,v) = X.TnT()

# Nonlinear:

maxit = int(1e7)
tol2 = 1e-8
regb = 1e-12

B_psi = ngs.GridFunction(X)

B = B_psi.components[0]
psi = B_psi.components[1]

normB = ngs.sqrt(B*B + regb)

cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - Hs*B, mesh)
    return res

cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()
rhs = ngs.LinearForm((cf_rhs*B*q -ngs.grad(psi)*q -Hs*q +B*ngs.grad(v))*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs

Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1], B[0]*B[2],
              B[1]*B[0], B[1]*B[1], B[1]*B[2],
              B[2]*B[0], B[2]*B[1], B[2]*B[2]), dims = (3,3))

fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

# K = ngs.BilinearForm(X, condense = True)
K_iter = ngs.BilinearForm(X)
K_iter += (cf_iter*p*q - ngs.grad(u)*q + p*ngs.grad(v))*ngs.dx
# C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [9]:
print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,X.ndof, "DOFs.\n ")

with ngs.TaskManager(): B.Set(ngs.CF((0,0,0)))
with ngs.TaskManager(): psi.Set(ngs.CF((0)))

du = ngs.GridFunction(X)
uo = ngs.GridFunction(X)
wo = 1e12

for it in range(1,maxit+1):
    
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    with ngs.TaskManager():
        iterativeSolver = CGSolver(K_iter.mat, freedofs = X.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
        # iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-2,  maxiter = maxit)
        du.vec.data = iterativeSolver * dw.vec
    
    if len(iterativeSolver.residuals) == maxit: print("... reached maxit!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm

    # wn = 1e12
    # if abs(wn-w) < tol2:
    if nrm/nrm0 < tol2:
        print("converged to desired tolerance")
        break
    elif abs(wo-w) < tol2*1e-2:
        print("stopped early due to stagnation")
        break
    else:
        # linesearch
        uo.vec.data = B_psi.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            B_psi.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.1*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                B_psi.vec.data = uo.vec.data

Using 3D mesh with ne= 21022 elements and nv= 3727 points and  66793 DOFs.
 
Iter:  1 | assem : 0.18s | CG took 0.54s with 1575 iterations | alpha : 1.00 | energy = 0.0000000000 | relres = 1.00e+00 |


KeyboardInterrupt: 

In [ ]:
L2 = ngs.VectorL2(mesh, dim = 3, order=2)
Bl2 = ngs.GridFunction(L2)
with ngs.TaskManager(): Bl2.Set(B)
Bl2np = Bl2.vec.FV().NumPy()
HCurl.ndof

import matplotlib.pyplot as plt
import numpy as np
max(abs(Bl2np))

NameError: name 'HCurl' is not defined

In [ ]:
Draw(B, mesh, clipping={"z":-1, "dist":0.064})

In [ ]:
B_psi.components

(<ngsolve.comp.ComponentGridFunction at 0x12cde8f50>,
 <ngsolve.comp.ComponentGridFunction at 0x12cde8fb0>)